In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('yelp_review.csv')

In [4]:
df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [5]:
#reading dataset with information about business(type)
data_business=pd.read_csv('yelp_business.csv')

In [6]:
data_business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [7]:
business=data_business[data_business['categories'].str.match('Restaurant')].copy()

In [8]:
# merging reviews and businnes based on business_id
f1=pd.merge(df, business, on='business_id')

In [9]:
f1.head()

,review_id,user_id,business_id,stars_x,date,text,useful,funny,cool,name,...,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,categories
0,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
1,VrhTbFhgUa4zmx6jfqp_hQ,gKYaocnFD8Zi6rMt_mT-cw,VR6GpWIda3SfvPC-lg9H3w,5,2015-01-31,Highly recommend checking this place out! Tuc...,1,0,0,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
2,XMJ_sHKVrtqJZ4IXA0I9wA,Vp_C7oIxeztbd2PchNBOHw,VR6GpWIda3SfvPC-lg9H3w,4,2011-10-23,This restaurant is part of the wave of trendy ...,1,0,4,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
3,MhA0-xaUx4o4Pl_GQUr_ZQ,tB6vkXCS_8PIh-T9g8t6IQ,VR6GpWIda3SfvPC-lg9H3w,5,2011-07-22,"It's in a sketchy area, but it's upscale and l...",2,0,1,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
4,ziRcRyoy96J1Wj_WN-pM2Q,i1XAHltBpZC5zP5skcwVGw,VR6GpWIda3SfvPC-lg9H3w,4,2014-11-01,A local friend introduced me to this jewel res...,3,2,2,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian


Important features user_id, text, stars_x (rating from user)

In [25]:
review_df = f1[['user_id','name','stars_x', 'text','city', 'categories']]

In [18]:
review_df.shape

(897135, 6)

In [19]:
review_df.head()

,user_id,name,stars_x,text,city,categories
0,bv2nCi5Qv5vroFiqKGopiw,"""Tuck Shop""",5,Small unassuming place that changes their menu...,Montréal,Restaurants;Canadian (New);Italian
1,gKYaocnFD8Zi6rMt_mT-cw,"""Tuck Shop""",5,Highly recommend checking this place out! Tuc...,Montréal,Restaurants;Canadian (New);Italian
2,Vp_C7oIxeztbd2PchNBOHw,"""Tuck Shop""",4,This restaurant is part of the wave of trendy ...,Montréal,Restaurants;Canadian (New);Italian
3,tB6vkXCS_8PIh-T9g8t6IQ,"""Tuck Shop""",5,"It's in a sketchy area, but it's upscale and l...",Montréal,Restaurants;Canadian (New);Italian
4,i1XAHltBpZC5zP5skcwVGw,"""Tuck Shop""",4,A local friend introduced me to this jewel res...,Montréal,Restaurants;Canadian (New);Italian


In [20]:
type(review_df['categories'])

pandas.core.series.Series

In [22]:
#exporting csv
review_df.to_csv('example.csv')

In [27]:
#number of restaurant
len(review_df['name'].unique().tolist())

13819

In [28]:
#number of cities
len(review_df['city'].unique().tolist())

553

In [29]:
#number of comments per restaurant, restaurant chain increase numbre  of comments
review_df['name'].value_counts()

"MGM Grand Hotel"                                3445
"McDonald's"                                     3284
"Hash House A Go Go"                             3051
"SkinnyFATS"                                     2705
"Egg & I"                                        2594
"Pizzeria Bianco"                                2534
"Chipotle Mexican Grill"                         2519
"Yardbird Southern Table & Bar"                  2361
"Rollin Smoke Barbeque"                          2320
"Oyster Bar"                                     2293
"Monta Ramen"                                    2290
"In-N-Out Burger"                                2246
"Yama Sushi"                                     2242
"SUSHISAMBA - Las Vegas"                         2100
"Eat."                                           1940
"Pampas Churrascaria Brazilian Grille"           1909
"Red Robin Gourmet Burgers"                      1884
"Smashburger"                                    1858
"Oregano's Pizza Bistro"    

In [30]:
#number of comments per city. 
review_df['city'].value_counts()

Las Vegas                  244973
Phoenix                     89916
Toronto                     88926
Scottsdale                  44855
Charlotte                   42736
Pittsburgh                  35664
Montréal                    28229
Tempe                       26410
Chandler                    23518
Henderson                   20776
Mesa                        18838
Cleveland                   17887
Gilbert                     14433
Madison                     14247
Glendale                    11832
Mississauga                 10647
Markham                      8873
Peoria                       7910
Edinburgh                    7243
North Las Vegas              7112
Stuttgart                    5541
Champaign                    5076
Richmond Hill                4886
Scarborough                  4248
North York                   3550
Goodyear                     3486
Surprise                     3425
Vaughan                      3060
Concord                      2973
Huntersville  

In [34]:
#selecting comments from Las Vegas
review_Vegas=review_df[review_df['city']=="Las Vegas"]

In [35]:
review_Vegas.shape

(244973, 6)

In [36]:
review_Vegas.head()

,user_id,name,stars_x,text,city,categories
4298,tL2pS5UOmN6aAOi3Z-qFGg,"""McDonald's""",3,I've stopped in here several times. It's alway...,Las Vegas,Restaurants;Fast Food;Food;Burgers
4299,Tgo-NHUmzX8SeVad5_-vug,"""McDonald's""",5,Best customer service in town! Super polite te...,Las Vegas,Restaurants;Fast Food;Food;Burgers
4300,vSzquh2Ym99tpksfpWH2kg,"""McDonald's""",1,I would give no stars if I could. Ordered a va...,Las Vegas,Restaurants;Fast Food;Food;Burgers
4301,LnC93etRS5OZG-VDZHdSkw,"""McDonald's""",2,I know its just McDonalds not the White House ...,Las Vegas,Restaurants;Fast Food;Food;Burgers
4302,2elkzAuFQdvCz0_K6MP4Dw,"""McDonald's""",5,We had an amazing experience here. It's very n...,Las Vegas,Restaurants;Fast Food;Food;Burgers


In [37]:
#exporting csv
review_Vegas.to_csv('Vegas.csv')

In [38]:
review_Vegas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244973 entries, 4298 to 897070
Data columns (total 6 columns):
user_id       244973 non-null object
name          244973 non-null object
stars_x       244973 non-null int64
text          244973 non-null object
city          244973 non-null object
categories    244973 non-null object
dtypes: int64(1), object(5)
memory usage: 13.1+ MB


# Data Cleaning

In [39]:
review_arr =  review_Vegas.as_matrix()

C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [40]:
review_arr.shape

(244973, 6)

In [41]:
clean_ar = np.empty(shape=(review_arr.shape), dtype=object)

In [42]:
type(clean_ar[0][1])

NoneType